In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
# from dataFilter import butter_highpass_filter, butter_lowpass_filter

In [1]:
#function for low/high pass filter
from scipy.signal import butter, lfilter, freqz

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y    

In [2]:
#function for reading database, return a dataframe, return a dataframe
def readDB(csv_database):
    sql = 'select * from sensorProcessed' #sql need to be executed. 
    df = pd.read_sql_query(sql, csv_database) # read data as dataframe
    return df

In [3]:
#set credentials file for plotly(username & API key)
import plotly
username = 'mengyizhou95' #username registered on Ploty
api_key = 'X5DOiJATuUqPbsabrbxA' #the api key plotly asigned
plotly.tools.set_credentials_file(username=username, api_key=api_key)

In [4]:
#set the conversion ratio from min to ms
timeConversionVal = 60000

In [5]:
#function for trans time from min to ms
def MinToMS(time, df):
    time = time * timeConversionVal
    time = df.time.iloc[(df['time']-time).abs().argsort()[:1]] #find the closest time on table
    return time.values[0]

In [6]:
#make the flapper data to voltage
def transVolt(df):
    for i in range(8):
        df['flapper' + str(i)] = 1000 / df['flapper' + str(i)] - 10

In [7]:
#function for generate flapper Chart
def flapperChart(df, start=None, end=None, filename='flapperChart'):
    #if we dont have start & end time point, assign the first & last time from data
    #if we have, make it to micro seconds
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    #get the index of start & end from time
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #generate chart
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=df.flapper0[startIndex: endIndex],
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper1[startIndex: endIndex],
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper2[startIndex: endIndex],
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper3[startIndex: endIndex],
        name = 'flapper3',
        line = dict(
            color = ('rgb(0, 220, 255)'),
            width = 1)
    )
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper4[startIndex: endIndex],
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper5[startIndex: endIndex],
        name = 'flapper5',
        line = dict(
            color = ('rgb(255, 127, 0)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper6[startIndex: endIndex],
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper7[startIndex: endIndex],
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Flapper sensor output'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate

In [8]:
#function for generate flapper Chart
def flapperProcessedChart(df, start=None, end=None, filename='flapperProcessedChart', freq=5):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
#     df = df[startIndex: endIndex]
    for i in range(8):
#         df['flapper' + str(i)] = butter_highpass_filter(butter_lowpass_filter(df['flapper' + str(i)], 5, 50, 5), 1, 50, 5)
        df['flapper' + str(i)] = butter_lowpass_filter(df['flapper' + str(i)], freq, 50, 5)
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=df.flapper0[startIndex: endIndex] + 40,
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper1[startIndex: endIndex] + 30,
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper2[startIndex: endIndex] + 20,
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper3[startIndex: endIndex] + 10,
        name = 'flapper3',
        line = dict(
            color = ('rgb(0, 220, 255)'),
            width = 1)
    )
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper4[startIndex: endIndex],
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper5[startIndex: endIndex] - 20,
        name = 'flapper5',
        line = dict(
            color = ('rgb(255, 127, 0)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper6[startIndex: endIndex] - 30,
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper7[startIndex: endIndex] - 40,
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]

    layout = dict(title = filename,
                       xaxis = dict(title = 'time(minutes)'),
                       yaxis = dict(title = 'Flapper sensor output'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate

In [9]:
#function for generate diameter chart
def sensorChart(df, start=None, end=None, filename='sensorChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0] # get index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.bend[startIndex: endIndex],
        name = 'bend',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.light[startIndex: endIndex],
        name = 'light',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )

    data = [trace0, trace1]
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time'),
                       yaxis = dict(title = 'sensor output'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [10]:
#function for debouncing orientation
def orientationTrans(dfOrientation, start, end, threshold, df):
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index
    endIndex = df.index[df['time'] == end].tolist()[0]
    prev = dfOrientation[startIndex]
    
    for i in range(startIndex, endIndex+1):
        diff = dfOrientation[i] - prev
        prev = dfOrientation[i]
        if(diff > threshold or diff < -threshold):
            if(diff > 0):
                dfOrientation[i] = threshold * 2 - diff
            else:
                dfOrientation[i] = -(threshold * 2 + diff)
        else:
            dfOrientation[i] = diff
    return dfOrientation

In [11]:
#function for generating orientation chart
def orientationChartTrans(df, start=None, end=None, filename='orientationChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    dfOrientationX = orientationTrans(df.orientationX, start, end, 180, df)
    dfOrientationY = orientationTrans(df.orientationY, start, end, 90, df)
    dfOrientationZ = orientationTrans(df.orientationZ, start, end, 180, df)

    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=dfOrientationX[startIndex: endIndex],
        name = 'orientationX',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=dfOrientationY[startIndex: endIndex],
        name = 'orientationY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=dfOrientationZ[startIndex: endIndex],
        name = 'orientationZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2]
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Orientation sensor output'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html')

In [12]:
#function for generating orientation chart
def orientationChart(df, start=None, end=None, filename='orientationChartOriginal'):
     
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    dfOrientationX = df.orientationX
    dfOrientationY = df.orientationY
    dfOrientationZ = df.orientationZ

    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=dfOrientationX[startIndex: endIndex],
        name = 'orientationX',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=dfOrientationY[startIndex: endIndex],
        name = 'orientationY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=dfOrientationZ[startIndex: endIndex],
        name = 'orientationZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )

    data = [trace0, trace1, trace2]
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Orientation sensor output'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [13]:
#function for generating acceleration chart
def accChart(df, start=None, end=None, filename='accChart'):

    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)

    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    v = (df.accX[startIndex: endIndex] ** 2 + df.accY[startIndex: endIndex] ** 2 + df.accZ[startIndex: endIndex] ** 2) ** (1/2)
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.accX[startIndex: endIndex],
        name = 'accX',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.accY[startIndex: endIndex],
        name = 'accY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.accZ[startIndex: endIndex],
        name = 'accZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=v,
        name = 'accSum',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )

    data = [trace0, trace1, trace2, trace3]
    
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Acceleration'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [14]:
#function for generating compass chart
def compassChart(df, start=None, end=None, filename='compassChart'):
         
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
        
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.compassX[startIndex: endIndex],
        name = 'compassX',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.compassY[startIndex: endIndex],
        name = 'compassY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.compassZ[startIndex: endIndex],
        name = 'compassZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    
    x = df.compassX
    y = df.compassY
    z = df.compassZ

    v = (x ** 2 + y ** 2 + z ** 2) ** (1/2)
    
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=v[startIndex: endIndex],
        name = 'compassSum',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )

    data = [trace0, trace1, trace2, trace3]
    

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'compass sensor output'))
                 

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [15]:
#function for generating compass chart
def rotationalSpeedChart(df, start=None, end=None, filename='rotationalSpeedChart'):
         
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
        
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = df.rotationalSpeedX[startIndex: endIndex],
        name = 'rotationalSpeedX',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = df.rotationalSpeedY[startIndex: endIndex],
        name = 'rotationalSpeedY',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = df.rotationalSpeedZ[startIndex: endIndex],
        name = 'rotationalSpeedZ',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x = df.time[startIndex: endIndex]/timeConversionVal,
        y = (df.rotationalSpeedX ** 2 + df.rotationalSpeedY ** 2 + df.rotationalSpeedZ ** 2) ** (1/2),
        name = 'rotationalSpeed',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1
        )
    )

    data = [trace0, trace1, trace2, trace3]
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Rotation Speed'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [16]:
# csv_database = 'sqlite:///602.db' #database name
# df = readDB(csv_database) #read database

In [87]:
run = '03'
csv_database = 'sqlite:///Dat58.db' 
df = readDB(csv_database)

In [88]:
flapperChart(df, filename='test' + str(run))

In [89]:
df = pd.read_csv('../dataset/Cambridge_trim.csv')

In [31]:
run = '01'
# df = pd.read_csv('/Users/meng/Documents/WatchTower/rawData/Wise County/processed Data/Wise_Run' + str(run) + '_Trimed.csv')
df = pd.read_csv('./Cambridge_Run01_Trimed.csv')

In [39]:
accChart(df, filename='Acc sensor output in Wise run' + str(run))

In [90]:
compassChart(df, filename='Magnetic sensor output in Wise run' + str(run))

In [35]:
sensorChart(df, filename='Sense Output in SanAntonio')

In [28]:
rotationalSpeedChart(df, filename='rotation speed output in Wise run' + str(run))

In [25]:
accChart(df, filename='Acc sensor output in Wise run' + str(run))

In [37]:
compassChart(df, filename='Magnetic sensor output in Wise run' + str(run))

In [27]:
orientationChart(df, filename='Orientation Output in SanAntonio)')

In [64]:
accChart(df, filename='Acc sensor output in Cambridge run' + str(run))
compassChart(df, filename='Magnetic sensor output in Cambridge run' + str(run))
rotationalSpeedChart(df, filename='rotation speed output in Cambridge run' + str(run))

In [83]:
flapperChart(df, filename='../charts/Flapper sensor output in Cambridge run' + str(run))
accChart(df, filename='../charts/Acc sensor output in Cambridge run' + str(run))
compassChart(df, filename='../charts/Magnetic sensor output in Cambridge run' + str(run))
rotationalSpeedChart(df, filename='../charts/rotation speed output in Cambridge run' + str(run))
orientationChart(df, filename='../charts/Orientation Output in Cambridge run' + str(run))